In [ ]:
import os
import sys
import timeit
from itertools import permutations

import numpy as np
from numpy.random import rand

import pandas as pd

from cec2013lsgo.cec2013 import Benchmark

from niapy.task import Task
from niapy.problems import Problem

from niapy.algorithms.algorithm import (
    AnalysisAlgorithm,
    OptimizationAlgorithm
)
from niapy.algorithms.analysis import (
    RecursiveDifferentialGrouping,
    RecursiveDifferentialGroupingV2,
    RecursiveDifferentialGroupingV3,
    EfficientRecursiveDifferentialGrouping,
    ThreeLevelRecursiveDifferentialGrouping
)
from niapy.algorithms.basic import (
    ParticleSwarmAlgorithm,
    ParticleSwarmOptimization,
    CenterParticleSwarmOptimization,
    MutatedParticleSwarmOptimization,
    MutatedCenterParticleSwarmOptimization,
    ComprehensiveLearningParticleSwarmOptimizer,
    MutatedCenterUnifiedParticleSwarmOptimization,
    OppositionVelocityClampingParticleSwarmOptimization,
)
from ccalgorithm import CooperativeCoevolution

In [ ]:
class CEC2013lsgoTask(Task):
    Name:list[str] = ['CEC2013lsgo']
    
    def __init__(self, no_fun:int, *args:list, **kwargs:dict) -> None:
        if 1 < no_fun > 15: raise Exception('Function between 1 and 15!!!')
        bench = Benchmark()
        info = bench.get_info(no_fun)
        max_evals = 3e6
        fun_fitness = bench.get_function(no_fun)
        
        class CEC2013lsgoProblem(Problem):
            def __init__(self, *args:list, **kwargs:dict) -> None:
                kwargs.pop('dimension', None), kwargs.pop('lower', None), kwargs.pop('upper', None)
                super().__init__(dimension=info['dimension'], lower=info['lower'], upper=info['upper'], *args, **kwargs)
        
            def _evaluate(self, x):
                return fun_fitness(x)

        kwargs.pop('problem', None), kwargs.pop('optimization_type', None), kwargs.pop('lower', None), kwargs.pop('upper', None), kwargs.pop('dimension', None), kwargs.pop('max_evals', None)
        super().__init__(problem=CEC2013lsgoProblem(), max_evals=max_evals, *args, **kwargs)

    def get_mesures(self) -> list[float]:
        r = [self.fitness_evals[0][1], self.fitness_evals[0][1], self.fitness_evals[0][1]]
        for e in self.fitness_evals:
            if e[0] > 120000: break
            else: r[0] = e[1]
        for e in self.fitness_evals:
            if e[0] > 600000: break
            else: r[1] = e[1]
        for e in self.fitness_evals:
            if e[0] > 3000000: break
            else: r[2] = e[1]
        return r

# Test bechmark functions

In [ ]:
def run_test_func(no_fun:int, max_evals:int = 1e6) -> None:
    if 1 < no_fun > 15: raise Exception('Function between 1 and 15!!!')
    bench = Benchmark()
    info = bench.get_info(no_fun)
    fun_fitness = bench.get_function(no_fun)
    start = timeit.default_timer()
    i = 0
    while i < max_evals:
        x = info['lower'] + rand(info['dimension']) * (info['upper'] - info['lower'])
        f = fun_fitness(x)
        i += 1
    end = timeit.default_timer()
    print('Time of execution for f%d for %d evals = %fs' % (no_fun, max_evals, end - start))

In [ ]:
for i in range(1, 16): run_test_func(i, 3e6)

# Grouping results

In [ ]:
def no_seps(a:list) -> int:
    s = 0
    for e in a:
        if len(e) > 1: continue
        s += 1
    return s

In [ ]:
def no_groups(a:list) -> int:
    s = 0
    for e in a:
        if len(e) == 1: continue
        s += 1
    return s

In [ ]:
def make_b_list(best: list) -> list:
    b, s = [], []
    for e in best:
        if len(e) == 1: s.append(e[0])
        else: b.append(e)
    if len(s) > 0: b.append(s)
    return b

In [ ]:
def run_grouping_cec2013(algo:AnalysisAlgorithm, id:int = 1, no_fun:int = 1, **algparams) -> None:
    algo.set_parameters(**algparams)
    task = CEC2013lsgoTask(no_fun=no_fun)
    start = timeit.default_timer()
    best = algo.run(task)
    stop = timeit.default_timer()
    acc, b = 0, make_b_list(best)
    if os.path.isfile('inputdata/F%d-s.txt' % no_fun):
        g_s = list(pd.read_csv('inputdata/F%d-s.txt' % no_fun, header=None).iloc[:, 0])
        cs = [e - 1 for e in list(pd.read_csv('inputdata/F%d-p.txt' % no_fun, header=None).iloc[0, :])]
        s, g = 0, []
        for e in g_s:
            g.append(cs[s:s + e])
            s += e
        g.append(cs[s:])
        if len(cs) < 1000: g.append(list(set(range(0, 1000)) - set(cs)))
        for p in permutations(g):
            t = 0
            for i, e in enumerate(p):
                if i >= len(b): break
                t = t + len(set(b[i]) & set(e))
                if t > acc: acc = t
        acc = acc / 1000
    else:
        acc = len(set(range(1000)) & set(b[0])) / 1000
    if not os.path.exists('./%s.cec2013lso.%d.csv' % (algo.Name[1], no_fun)):
        with open('%s.cec2013lso.%d.csv' % (algo.Name[1], no_fun), 'w') as csvfile:
            csvfile.write('seed, no_groups, no_seps, accuracy, evals, time, allgroups\n')
    with open('%s.cec2013lso.%d.csv' % (algo.Name[1], no_fun), 'a') as csvfile:
        csvfile.write('%d, %d, %d, %f, %d, %f, "%s"\n' % (id, no_groups(best), no_seps(best), acc, task.evals, stop - start, best))

## RecursiveDifferentialGrouping

In [ ]:
for i in range(1, 16):
    for j in range(1):
        print('F%d-R%d' % (i, j))
        run_grouping_cec2013(RecursiveDifferentialGrouping(seed=j), j, i)

## RecursiveDifferentialGrouping v2

In [ ]:
for i in range(1, 16):
    for j in range(51):
        print('F%d-R%d' % (i, j))
        run_grouping_cec2013(RecursiveDifferentialGroupingV2(seed=j), j, i)

## RecursiveDifferentialGrouping v3

In [ ]:
for i in range(1, 16):
    for j in range(51):
        print('F%d-R%d' % (i, j))
        run_grouping_cec2013(RecursiveDifferentialGroupingV3(seed=j), j, i)

## EfficientRecursiveDifferentialGrouping

In [ ]:
for i in range(1, 16):
    for j in range(51):
        print('F%d-R%d' % (i, j))
        run_grouping_cec2013(EfficientRecursiveDifferentialGrouping(seed=j), j, i)

## ThreeLevelRecursiveDifferentialGrouping

In [ ]:
for i in range(1, 16):
    for j in range(51):
        print('F%d-R%d' % (i, j))
        run_grouping_cec2013(ThreeLevelRecursiveDifferentialGrouping(seed=j), j, i)

# PSO results

In [ ]:
def run_cec2013(opt:type[OptimizationAlgorithm] = ParticleSwarmAlgorithm, no_fun:int = 1, seed:int = 1) -> None:
    algo = opt(seed=seed)
    task = CEC2013lsgoTask(no_fun=no_fun)
    start = timeit.default_timer()
    res = algo.run(task)
    stop = timeit.default_timer()
    if not os.path.exists('%s.%s.%d.csv' % (algo.Name[1], task.Name[0], no_fun)):
        with open('%s.%s.%d.csv' % (algo.Name[1], task.Name[0], no_fun), 'w') as csvfile:
            csvfile.write('seed, f1, f2, f3, time\n')
    with open('%s.%s.%d.csv' % (algo.Name[1], task.Name[0], no_fun), 'a') as csvfile:
        f1, f2, f3 = task.get_mesures()
        csvfile.write('%d, %f, %f, %f, %f\n' % (seed, f1, f2, f3, stop - start))

In [ ]:
arg_no_fun = int(input('Function no.:'))
arg_seed = int(input('Seed no.:'))
arg_opt_alg = int(input('Algorithm no.:'))
opt_alg = ParticleSwarmAlgorithm
if arg_opt_alg == 2: opt_alg = ParticleSwarmOptimization
elif arg_opt_alg == 3: opt_alg = CenterParticleSwarmOptimization
elif arg_opt_alg == 4: opt_alg = MutatedParticleSwarmOptimization
elif arg_opt_alg == 5: opt_alg = MutatedCenterParticleSwarmOptimization
elif arg_opt_alg == 6: opt_alg = OppositionVelocityClampingParticleSwarmOptimization
elif arg_opt_alg == 7: opt_alg = ComprehensiveLearningParticleSwarmOptimizer
else: opt_alg = ParticleSwarmAlgorithm
np.set_printoptions(linewidth=np.inf)
run_cec2013(opt=opt_alg, no_fun=arg_no_fun, seed=arg_seed)

# PSO with grouping

In [ ]:
def run_cc_cec2013(decomp:type[AnalysisAlgorithm] = RecursiveDifferentialGroupingV3, opt:type[OptimizationAlgorithm] = ParticleSwarmAlgorithm, no_fun:int = 1, seed:int = 1) -> None:
    algo = CooperativeCoevolution(decomp(seed=seed), opt, seed=seed)
    task = CEC2013lsgoTask(no_fun=no_fun)
    start = timeit.default_timer()
    res = algo.run(task)
    stop = timeit.default_timer()
    if not os.path.exists('%s.%s.%s.%d.csv' % (algo.decompozer.Name[1], algo.toptimizer.Name[1], task.Name[0], no_fun)):
        with open('%s.%s.%s.%d.csv' % (algo.decompozer.Name[1], algo.toptimizer.Name[1], task.Name[0], no_fun), 'w') as csvfile:
            csvfile.write('seed, f1, f2, f3, time\n')
    with open('%s.%s.%s.%d.csv' % (algo.decompozer.Name[1], algo.toptimizer.Name[1], task.Name[0], no_fun), 'a') as csvfile:
        f1, f2, f3 = task.get_mesures()
        csvfile.write('%d, %f, %f, %f, %f\n' % (seed, f1, f2, f3, stop - start))

In [ ]:
arg_no_fun = int(input('Function no.:'))
arg_seed = int(input('Seed no.:'))
arg_opt_alg = int(input('Algorithm no.:'))
arg_decomp_alg = int(input('Decomposer no.:'))
opt_alg = ParticleSwarmAlgorithm
if arg_opt_alg == 2: opt_alg = ParticleSwarmOptimization
elif arg_opt_alg == 3: opt_alg = CenterParticleSwarmOptimization
elif arg_opt_alg == 4: opt_alg = MutatedParticleSwarmOptimization
elif arg_opt_alg == 5: opt_alg = MutatedCenterParticleSwarmOptimization
elif arg_opt_alg == 6: opt_alg = OppositionVelocityClampingParticleSwarmOptimization
elif arg_opt_alg == 7: opt_alg = ComprehensiveLearningParticleSwarmOptimizer
else: opt_alg = ParticleSwarmAlgorithm
decomp_alg = RecursiveDifferentialGroupingV3
if arg_decomp_alg == 1: RecursiveDifferentialGrouping
elif arg_decomp_alg == 2: RecursiveDifferentialGroupingV2
elif arg_decomp_alg == 3: RecursiveDifferentialGroupingV3
elif arg_decomp_alg == 4: EfficientRecursiveDifferentialGrouping
elif arg_decomp_alg == 5: ThreeLevelRecursiveDifferentialGrouping
else: decomp_alg = RecursiveDifferentialGroupingV3
np.set_printoptions(linewidth=np.inf)
run_cc_cec2013(decomp=RecursiveDifferentialGroupingV3, opt=ParticleSwarmAlgorithm, no_fun=arg_no_fun, seed=arg_seed)